<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/image/IDSN-logo.png" width="400"> </a>


# Part 3: Concrete Crack Image Classification — Model training

**Short description:**  
This notebook performs transfer learning using a ResNet50 backbone (pretrained on ImageNet) to classify concrete crack images. It extracts a zip dataset, creates Keras `ImageDataGenerator` flows for training/validation, builds a model with ResNet50 base + a classification head, trains the model, and saves the trained model.

**Dataset:**

Download the dataset from: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/concrete_data_week3.zip

**Objectives**
- Extract the zipped dataset and create a working directory  with `zipfile`. 
- Create `ImageDataGenerator` flows for training and validation with `preprocess_input` suited for ResNet.  
- Build a Keras `Sequential` model using a frozen ResNet50 base (pooling='avg') and a dense softmax head for `num_classes`.  
- Train the model and save it to disk (`classifier_resnet_model.h5`).  

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & execution instructions  
2. Unzip dataset and inspect directories 
3. Define global constants
4. ImageDataGenerator & directory flows 
5. Model construction (ResNet50 base + head)  
6. Training and saving the model  
7. Notes & reproducibility


## 1) Dependencies & execution instructions

This section installs and imports required Python packages.  

**Recommended local execution steps:**

1. Create and activate a Python virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate` (macOS / Linux) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Launch Jupyter Notebook:
   - `jupyter notebook`
4. Open this notebook and run cells top-to-bottom.

**Note:** Make sure to have the `concrete_data_week3.zip` in the root directory.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
import zipfile
from pathlib import Path

## 2) Unzip dataset and inspect directories

Extracts `concrete_data_week3.zip` into `concrete_data_week3/`. After extraction, confirm the structure contains `train/` and `valid/` subdirectories with class subfolders inside (one folder per class).

In [ ]:
zip_path = Path("concrete_data_week3.zip")

extract_dir = Path("concrete_data_week3")
extract_dir.mkdir(exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

**Important Note:** 

There are thousands and thousands of images in each folder, so please don't attempt to double click on the *negative* and *positive* folders. This may consume all of your memory and you may end up with a **50** error. So please **DO NOT DO IT**.


## 3) Define global constants

Define constants:

1. There are two clases: `Positive` and `Negative`. `num_classes` is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, the images have to be resized from (227 x 227) to (224 x 224).
3. Perform training and validation with batches of 100 images.

In [ ]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## 4) ImageDataGenerator & directory flows

We create `ImageDataGenerator` with `preprocess_input` (ResNet50). Then we create `train_generator` and `validation_generator` via `flow_from_directory()` using the target size (224×224) and categorical class mode.


Set the **preprocessing_function** argument to *preprocess_input* in order to preprocess the images the same way the images used to train ResNet50 model were processed.


In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [ ]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

## 5) Model construction (ResNet50 base + head)

We create a `Sequential` model and add a ResNet50 base (`include_top=False`, `pooling='avg'`, `weights='imagenet'`) followed by a Dense softmax classification head for `num_classes`. The ResNet base is frozen (`trainable = False`) before compiling.


In [ ]:
model = Sequential()

In [ ]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Define output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.


In [ ]:
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.layers

The model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is the Dense layer defined above.


In [ ]:
model.layers[0].layers

Since the ResNet50 model has already been trained, the ResNet part won't be retrained, only the dense output layer will be trained.


In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

## 6) Training and saving the model 

Compile the model with `adam` optimizer and `categorical_crossentropy` loss, train for `num_epochs` with steps determined by the generators, and then save the trained model as `classifier_resnet_model.h5`.


Compile the model using the **adam** optimizer.


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [ ]:
model.save('classifier_resnet_model.h5')

## 6) Notes & reproducibility

- `keras` can be the standalone package or `tensorflow.keras`. If you use TensorFlow 2.x, prefer `from tensorflow.keras...` and install `tensorflow`.  
- If the machine has a GPU, ensure the correct TensorFlow/PyTorch GPU build is installed to accelerate training.  
- If dataset sizes are large, adjust `batch_size_training` and `batch_size_validation` to fit memory / GPU capacity.


## About the Authors:

 [Alex Aklson](https://www.linkedin.com/in/aklson/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0321ENSkillsNetwork951-2022-01-01)


This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week2_LAB1).



## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2020-09-18  | 2.0  | Shubham  |  Migrated Lab to Markdown and added to course repo in GitLab |
| 2023-01-03  | 3.0  | Artem |  Updated the file import section|



<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
